# Simulate a Neural Reservoir

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import igraph as ig
from matplotlib.animation import FuncAnimation
from tqdm import tqdm
from config import DATA_PATH

folder = DATA_PATH / 'experiments' 

def simulate_reservoir(num_neurons, connection_probability, time_steps, initially_active_ratio):
    """
    Create a bunch of neurons that are randomly connected to each other.
    Neurons can have two states: on and off.

    Args:
        num_neurons (int): Number of neurons in the reservoir.
        connection_probability (float): Probability of connection between any two neurons.
        time_steps (int): Number of time steps to simulate.
    """
    
    connection_matrix = np.random.random((num_neurons, num_neurons)) < connection_probability
    neuron_state_time_matrix = np.zeros((time_steps, num_neurons), dtype=int)
    
    # randomly activate 10% of neurons initially
    neuron_state_time_matrix[0] = (np.random.random(num_neurons) < initially_active_ratio).astype(int)

    # simulate network activity over time
    for t in range(1, time_steps):
        # for each neuron, check if any of its connected neighbors were active at t-1
        for i in range(num_neurons):
            # get all neurons that connect to neuron i
            neighbors = np.where(connection_matrix[:, i])[0]
            # check if any neighbors were active at previous time step
            if np.sum(neuron_state_time_matrix[t-1, neighbors] == 1) == 1:
                neuron_state_time_matrix[t, i] = 1
    
    return neuron_state_time_matrix, connection_matrix

def animate_reservoir(neuron_state_time_matrix, connection_matrix, file_name):
    """
    Animate the network activity over time.
    Args:
        neuron_state_time_matrix (np.ndarray): Matrix of neuron states over time.
        connection_matrix (np.ndarray): Connection matrix of the reservoir.
        file_name (str): Name of the output GIF file.
    """
    time_steps = neuron_state_time_matrix.shape[0]

    # create graph
    g = ig.Graph.Adjacency((connection_matrix > 0).tolist(), mode='directed')
    layout = g.layout(layout='auto')
    # layout = g.layout_fruchterman_reingold()

    # plot graph
    fig, ax = plt.subplots(figsize=(10, 8))
    
    def update(frame):
        ax.clear()
        node_colors = ['lightgray' if state == 0 else 'red' for state in neuron_state_time_matrix[frame]]
        ig.plot(
            g, 
            target=ax,
            layout=layout, 
            vertex_color=node_colors,
            vertex_size=10,
            vertex_label_size=8,
            edge_width=0.5,
            edge_arrow_size=0.5,
            edge_arrow_width=0.2,
            margin=30
        )
        ax.set_title(f"Reservoir at Timestep {frame}/{time_steps}")
        ax.set_axis_off()
        return ax

    frames = tqdm(range(time_steps), desc="Rendering animation")
    ani = FuncAnimation(fig, update, frames=frames, interval=500, blit=False)
    ani.save(file_name, writer='pillow', fps=10, dpi=100)
    plt.close()

if __name__ == "__main__":
    num_neurons = 300
    connection_probability = 0.01
    time_steps = 100
    initially_active_ratio = 0.01
    neuron_state_time_matrix, connection_matrix = simulate_reservoir(num_neurons, connection_probability, time_steps, initially_active_ratio)

    animate_reservoir(neuron_state_time_matrix, connection_matrix, folder / 'reservoir.gif')